**Goals**:
* connect to the database
* query the tables in the database
* **Activity**: basic sql queries to explore the data
* create a dataset which contains the descriptions of the stock

In [1]:
# import the libraries needed

import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings('ignore')

### Conect to the database

In [2]:
# connect to database

dbname = "dev"
host = "redshift-cluster-1.c9gt5btzchps.eu-central-1.redshift.amazonaws.com"
port = 5439
user = ""
password = ""

# connect to redshift

conn = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

### Query the Tables 

Table 1: Online Retail History

In [3]:
query = """select *
            from bootcamp.online_transactions"""

query

'select *\n            from bootcamp.online_transactions'

In [4]:
online_retail = pd.read_sql(query, conn)



In [5]:
# write query to select online retail history data

query = """select *
          from bootcamp.online_transactions
"""

In [6]:
# use the read_sql function in pandas to read a query into a DataFrame

online_retail = pd.read_sql(query, conn)


In [7]:
# preview the data

online_retail.head(n = 3)

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom


In [8]:
# structure of the data

online_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [9]:
# size of the data

online_retail.shape

# 540,000 rows of of data and 7 columns

(541910, 7)

Table 2: Stock Description

In [10]:
query = """select *
          from bootcamp.stock_description
"""

stock_description = pd.read_sql(query, conn)

In [11]:
# preview the data

stock_description.head(n = 3)

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER


In [12]:
# structure of the data

stock_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [13]:
# size of the data

stock_description.shape

# 4,000 rows of of data and 2 columns

(3952, 2)

### Optional Activity: Basic Exploration of the data using SQL 

In [14]:
# Q1. How many rows (total number of items) does the online retail history dataset contain?

# write the query

query = """select count(*) as number_rows
          from bootcamp.online_transactions
        """

pd.read_sql(query, conn)

,number_rows
0,541910


In [15]:
# Q2. How many customers does the dataset contain?

# write the query

query = """select count(distinct customer_id) as number_customers
          from bootcamp.online_transactions
"""

pd.read_sql(query, conn)

,number_customers
0,4373


In [16]:
# Q3. How many invoices does the dataset contain?

# write the query

query = """select count(distinct Invoice) as number_invoices
          from bootcamp.online_transactions
"""

pd.read_sql(query, conn)

,number_invoices
0,25900


In [17]:
# Q4. When was the first and last invoice?

# write the query

query = """select min(invoice_date) as min_invoice_date,
                  max(invoice_date) as max_invoice_date
          from bootcamp.online_transactions
"""

pd.read_sql(query, conn)

# We have one year of invoices - ranging from 1st Dec 2010 to 9th Dec 20111

,min_invoice_date,max_invoice_date
0,2010-12-01 08:26:00,2011-12-09 12:50:00


In [62]:
# Q5. How many invoices did we have per month? 

query = """
    select
        cast(date_part(year, cast(invoice_date as Datetime)) as int) as invoice_year,
        cast(date_part(month, cast(invoice_date as Datetime)) as int) as invoice_month,
        count(distinct invoice) as number_invoices
    from bootcamp.online_transactions
    group by invoice_year, invoice_month
    order by invoice_year, invoice_month
        """

pd.read_sql(query, conn)

,invoice_year,invoice_month,number_invoices
0,2010,12,2025
1,2011,1,1476
2,2011,2,1393
3,2011,3,1983
4,2011,4,1744
5,2011,5,2162
6,2011,6,2012
7,2011,7,1927
8,2011,8,1737
9,2011,9,2327


In [19]:
# Q6. How many different types of stocks did the customers purchase?

query = """select count(distinct stock_code) as number_stocks
          from bootcamp.online_transactions
"""

pd.read_sql(query, conn)

,number_stocks
0,4070


In [65]:
# Q7. What was the most popular stock?

query = """select stock_code,
                  sum(quantity) as total_sold
          from bootcamp.online_transactions
          group by 1
          order by 2 desc
"""

pd.read_sql(query, conn).head(10)

,stock_code,total_sold
0,22197,56450
1,84077,53847
2,85099B,47363
3,85123A,38830
4,84879,36221
5,21212,36039
6,23084,30646
7,22492,26437
8,22616,26315
9,21977,24753


In [66]:
query = """select stock_code,
                  sum(quantity) as total_sold
           from bootcamp.online_transactions
           group by stock_code
           order by total_sold desc
           limit 1
"""
pd.read_sql(query, conn)

,stock_code,total_sold
0,22197,56450


In [67]:
# Q8. What is the description of the most popular stock?
# Query the table stock_description

query = """select *
           from bootcamp.stock_description
           where stock_code in (select stock_code
                              from (select stock_code,
                                    sum(quantity) as number_stocks
                              from bootcamp.online_transactions
                              group by 1
                              order by 2 desc)
                              limit 1
                              )
"""

pd.read_sql(query, conn)

# The most popular stock has two descriptions - one being a ?

,stock_code,description
0,22197,POPCORN HOLDER


In [68]:
# Q9. How many Stocks have ? as the Description 

query = """select count(*) as number_question_marks
           from bootcamp.stock_description
           where Description = '?'
"""

pd.read_sql(query, conn)

,number_question_marks
0,47


In [69]:
# Q10. How many Stocks does the "stock description" dataset contain?

query = """
          select count(distinct stock_code) as number_stocks
          from bootcamp.stock_description
"""

pd.read_sql(query, conn)

# We identified 4070 stocks in the dataset

,number_stocks
0,3905


In [70]:
query = """
          select count(distinct stock_code) as number_stocks
          from bootcamp.online_transactions
"""

pd.read_sql(query, conn)

,number_stocks
0,4070


### Aggregate the Online Retail History and Stock Description Dataset

In [71]:
query = """
        select t1.*,
               t2.Description
        from bootcamp.online_transactions as t1
        /* sub query that remove stocks with ? as the description */
        left join (select *
                   from bootcamp.stock_description
                   where description <> '?') as t2 on t1.stock_code = t2.stock_code
    
"""

online_history_w_description = pd.read_sql(query, conn)
online_history_w_description.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,84029E,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.
2,536366,22632,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom,HAND WARMER RED POLKA DOT
3,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BLUE COAT RACK PARIS FASHION
4,536367,22749,8,2010-12-01 08:34:00,3.75,u13047,United Kingdom,FELTCRAFT PRINCESS CHARLOTTE DOLL


In [72]:
# check shape of the data

online_history_w_description.shape

# 54,000 rows of data with 8 columns 

(541910, 8)

In [73]:
# confirm the shape of the data has the same number of rows as the online retail history dataset

online_retail.shape

(541910, 7)

In [74]:
# confirm there are no ? in the dataset

online_history_w_description[online_history_w_description.description == '?']

# Expected output: Blank DataFrame

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description


In [75]:
online_history_w_description.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [76]:
# remove rows of data where quantity less than 0

query = """
        select t1.*,
               t2.Description
        from bootcamp.online_transactions as t1
        /* remove stocks with ? as the description */
        left join (select *
                   from bootcamp.stock_description
                   where description <> '?') as t2 on t1.stock_code = t2.stock_code
         /*known data error where items have negative quantity*/
         where quantity > 0
"""

online_history_w_description = pd.read_sql(query, conn)
online_history_w_description.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom,HAND WARMER UNION JACK
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,YELLOW COAT RACK PARIS FASHION
3,536367,22748,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom,POPPY'S PLAYHOUSE KITCHEN
4,536367,22623,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BOX OF VINTAGE JIGSAW BLOCKS


In [77]:
# check shape of data

online_history_w_description.shape

(531286, 8)

In [78]:
# close any connections to the DB

conn.close()